In [1]:
from pulp import*

In [2]:
# definition of sets and parameters

# Factories, distribution centers and stores (sets)
Factory = ["Purple", "Blue", "Orange"]
DistributionCenter = ["Strawberry", "Blackberry"]
Stores = ["Alpha", "Beta", "Gamma"]

# Capacities
FCapacity = {"Purple": 6000,
             "Blue"  : 8000,
             "Orange": 10000}

# Demand of the stores
Demand = {"Alpha":5000,
          "Beta": 12000,
          "Gamma": 7000}

# Costs
CostFDC = {"Purple":{"Strawberry": 3,
                     "Blackberry": 4},
           "Blue": {"Strawberry": 5,
                    "Blackberry": 2},
           "Orange": {"Strawberry": 6,
                      "Blackberry": 3}}

CostDCS = {"Strawberry": {"Alpha": 2,
                          "Beta": 6,
                          "Gamma": 3},
           "Blackberry": {"Alpha": 5,
                          "Beta": 4,
                          "Gamma": 7}}

In [3]:
prob = LpProblem("EGManufacturing", LpMinimize)

In [4]:
X = LpVariable.dicts("x",
                     (Factory, DistributionCenter),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

Y = LpVariable.dicts("x",
                     (DistributionCenter, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

In [5]:
prob += lpSum(CostFDC[i][j]*X[i][j] \
              for i in Factory for j in DistributionCenter) + lpSum(CostDCS[j][k]*Y[j][k] \
                                                                                          for j in DistributionCenter for k in Stores)

In [6]:
# Constraints

for k in Stores:
    prob += lpSum(Y[j][k] for j in DistributionCenter) >= Demand[k]
    
for j in DistributionCenter:
    prob += lpSum(X[i][j] for i in Factory) >= lpSum(Y[j][k] for k in Stores)
    
for i in Factory:
    prob += lpSum(X[i][j] for j in DistributionCenter) <= FCapacity[i]

In [7]:
prob.solve()

1

In [9]:
print("The problem is ", LpStatus[prob.status])
print("The objective function value is ", value(prob.objective))
for v in prob.variables():
    if v.varValue > 0.1:
        print(v.name, "=", v.varValue)

The problem is  Optimal
The objective function value is  161000.0
x_Blackberry_Beta = 12000.0
x_Blue_Blackberry = 2000.0
x_Blue_Strawberry = 6000.0
x_Orange_Blackberry = 10000.0
x_Purple_Strawberry = 6000.0
x_Strawberry_Alpha = 5000.0
x_Strawberry_Gamma = 7000.0
